# Matrix testing

In [18]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

DB_URL = "sqlite:///../../bleau_info.db"

engine = create_engine(DB_URL, echo=False)

session = Session(engine)

In [21]:
import sys

sys.path.append("../../")

In [19]:
from scipy.sparse import load_npz

similarity_ascents = load_npz("../../similarity_ascent.npz")
similarity_style = load_npz("../../similarity_style.npz")
similarity_grade = load_npz("../../similarity_grade.npz")

In [ ]:
import numpy as np
from sqlalchemy import select

from models.boulder import Boulder


def recommend_boulders(
    input_boulders,
    top_n=5,
    ascent_weight=0.5,
    style_weight=0.25,
    grade_weight=0.25,
):

    ascents = similarity_ascents[:, input_boulders].sum(axis=1).A1
    style = similarity_style[:, input_boulders].sum(axis=1).A1
    grade = similarity_grade[:, input_boulders].sum(axis=1).A1

    # Remove input boulders from the recommendation
    ascents[input_boulders] = 0
    style[input_boulders] = 0
    grade[input_boulders] = 0

    sim_scores = (
        ascent_weight * ascents + style_weight * style + grade_weight * grade
    )

    best_boulders = np.argsort(-sim_scores)[:top_n]

    return best_boulders.tolist()


ids = recommend_boulders(
    [6735], top_n=20, ascent_weight=0.5, style_weight=0.25, grade_weight=0.25
)

boulders = session.execute(
    select(Boulder.name, Boulder.id).filter(Boulder.id.in_(ids))
).all()

boulder_dict = {
    boulder_id: boulder_name for boulder_name, boulder_id in boulders
}
boulder_names = [(boulder_dict[boulder_id], boulder_id) for boulder_id in ids]
display(boulder_names)

[('Ouroboros', 18099),
 ('Le Proueptologue (assis)', 3820),
 ('Extraction Divine (assis)', 22853),
 ('La Théorie des Jeux (assis)', 15826),
 ('Magneton', 25502),
 ('Respire', 1822),
 ('Contre Sens', 28315),
 ('Énergie Collective', 23530),
 ('Imothep', 3639),
 ('Tilt', 37152),
 ('Hallelujah', 26303),
 ('Realist', 22742),
 ('Le Plat du Chef (assis)', 33602),
 ('Déforestation', 11651),
 ('Free Hug (assis)', 13458),
 ('Kaïken', 25204),
 ('Miss Tick', 3823),
 ('Le Maître est Talon (assis)', 17857),
 ('Saïgon (assis)', 6208),
 ('Saï (assis)', 3068)]